<a href="https://colab.research.google.com/github/wjvlno/python_bootcamp/blob/main/session5_filled_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup & Loading the Titanic Dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml

# Load the Titanic dataset from scikit-learn
titanic = fetch_openml('titanic', version=1, as_frame=True)
titanic = titanic.frame

# view the features for training
titanic

In [ ]:
# drop some irrelevant columns
titanic.drop(['home.dest', 'body', 'boat', 'embarked'], axis = 1, inplace = True)

# Getting an Intuition for the Data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Handle missing values in the 'age' column
# titanic['age'].fillna(titanic['age'].median(), inplace=True)

# Create the plot
plt.figure(figsize=(15, 8))
ax = sns.kdeplot(titanic["age"][titanic.survived == '1'], color="darkturquoise", fill=True, label='Survived')
sns.kdeplot(titanic["age"][titanic.survived == '0'], color="lightcoral", fill=True, label='Died')

# Add labels and title
plt.legend()
plt.title('Density Plot of Age for Surviving Population and Deceased Population')
ax.set(xlabel='Age')
plt.xlim(-10, 85)

# Show the plot
plt.show()


In [ ]:
# Fare vs survival

plt.figure(figsize=(15,8))
ax = sns.kdeplot(titanic["fare"][titanic.survived == '1'], color="darkturquoise", fill=True)
sns.kdeplot(titanic["fare"][titanic.survived == '0'], color="lightcoral", fill=True)
plt.legend(['Survived', 'Died'])
plt.title('Density Plot of Fare for Surviving Population and Deceased Population')
ax.set(xlabel='Fare')
plt.xlim(-20,200)
plt.show()

In [ ]:
# Passenger class vs. survival

titanic['survived'] = titanic['survived'].astype(int)

plt.figure(figsize=(8,8))
sns.barplot(x='pclass', y='survived', data=titanic, color="darkturquoise")
plt.show()

In [ ]:
# Gender vs. survival

plt.figure(figsize=(8,8))
sns.barplot(x='sex', y='survived', data=titanic, color="aquamarine")
plt.show()

# Running a Vanilla ML Model

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


# Load the Titanic dataset from scikit-learn
titanic = fetch_openml('titanic', version=1, as_frame=True)
titanic = titanic.frame

titanic.drop(['home.dest', 'body', 'boat', 'embarked'], axis = 1, inplace = True)

# Specify numeric and categorical columns
numeric_features = titanic.select_dtypes(include=['int64', 'float64']).columns
categorical_features = titanic.select_dtypes(include=['object']).columns

# Define a preprocessor (more on this later...)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='median'), numeric_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ])

# Build a pipeline that preprocesses the data then fits the logistic classifier
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

# Separate the data into features (X) and target (y)
X = titanic.drop(columns='survived')
y = titanic['survived'].astype(int)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Fit the pipeline
pipeline.fit(X_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the model
print("Test Set Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Plot the confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Plot the ROC curve
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = roc_auc_score(y_test, y_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC Curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

# Building a Better Classifier

### Start with the data - preprocessing and handling missing data

In [ ]:
# Visualize missing data

plt.figure(figsize=(12, 8))
sns.heatmap(titanic.isnull(), cbar=False, cmap='viridis', yticklabels=False)
plt.title('Missing Values Heatmap in Titanic Dataset')
plt.xlabel('Columns')
plt.ylabel('Rows')
plt.show()

# quick for loop to print number of missing values in each column
for col in titanic.columns:
  print(f'{col}: # missing = {titanic[col].isnull().sum()}')



# Observation: a lot of missing age and cabin data, plus one missing fare

### Missing Age Values

In [ ]:
# compute overall median age
overall_median_age = titanic['age'].median()

# Calculate the median age for each combination of pclass and sex
median_ages = titanic.groupby(['sex', 'pclass'])['age'].median().reset_index()

# Plotting
plt.figure(figsize=(10, 6))
g = sns.catplot(x='pclass', y='age', hue='sex', kind='bar', data=median_ages, palette='viridis')

# add a dotted line for overall median age
plt.axhline(y=overall_median_age, color='red', linestyle='--')


# Set plot titles and labels
g.set_titles("{col_name} Passengers")
g.set_axis_labels("Passenger Class (pclass)", "Median Age")
g.fig.suptitle('Median Ages for Different "Pclass" and "Sex"', fontsize=12)

plt.show()

# print the median ages for these different groups
age_by_pclass_sex = titanic.groupby(['sex', 'pclass'])['age'].median()

for pclass in range(1, 4):
    for sex in ['female', 'male']:
        print('Median age of Pclass {} {}s: {}'.format(pclass, sex, age_by_pclass_sex[sex][pclass]))
print('Median age of all passengers: {}'.format(titanic['age'].median()))


# The overall median might be useful for interpolating the ages of
# female passengers in the 2nd passenger class, but would likely undershoot the
# ages of the 1st passenger class, and undershoot the 3rd passenger class

# A better approach is to interpolate with medians based on the different
# levels of "sex" and "pclass" present in the dataset

In [ ]:
# interpolate age using pclass and sex variables
age_by_pclass_sex = titanic.groupby(['sex', 'pclass'])['age'].median()

for pclass in range(1, 4):
    for sex in ['female', 'male']:
        print('Median age of Pclass {} {}s: {}'.format(pclass, sex, age_by_pclass_sex[sex][pclass]))
print()
print('Median age of all passengers: {}'.format(titanic['age'].median()))


In [ ]:
# Filling the missing values in Age with the medians of Sex and Pclass groups
titanic['age'] = titanic.groupby(['sex', 'pclass'])['age'].apply(lambda x: x.fillna(x.median())).values

### Adding a New Variable (Family Size)

In [ ]:
# sibsp = number of siblings/spouses aboard
# parch = number of parents/children

# let's combine these into a family size variable

titanic['family_size'] = titanic['sibsp'] + titanic['parch'] + 1

### Missing Fare Value

In [ ]:
# Handling the missing fare value

# Calculate the median age for each combination of pclass and sex
median_fares = titanic.groupby(['pclass', 'family_size'])['fare'].median().reset_index()

# Plotting
plt.figure(figsize=(10, 6))
g = sns.catplot(x='pclass', y='fare', hue='family_size', kind='bar', data=median_fares, palette='viridis')

plt.axhline(y = titanic['fare'].median(), color='red', linestyle='--')

# Set plot titles and labels
g.set_titles("{col_name} Passengers")
g.set_axis_labels("Passenger Class (pclass)", "Median Fare")
g.fig.suptitle('Median Fares for Different "Pclass" and "Family Size"', fontsize=12)

plt.show()

In [ ]:
# check the median fare for different combinations of 'pclass' and 'family_size'

fare_by_pclass_fsize = titanic.groupby(['pclass', 'family_size'])['fare'].median()

for pclass in range(1, 4):
    for fsize in sorted(titanic[titanic['pclass'] == pclass]['family_size'].unique()):
        print('Median fare for Pclass {} w/ family size {}: {}'.format(pclass, fsize, fare_by_pclass_fsize[pclass][fsize]))
    print()

print()
print('Median age of all passengers: {}'.format(titanic['fare'].median()))

In [ ]:
# What's the pclass and family size for the passenger with a missing fare?

titanic[titanic['fare'].isnull()]

# pclass = 3
# family_size = 1

# so we should interpolate this passenger's fare with 7.8542

In [ ]:
titanic['fare'] = titanic['fare'].fillna(titanic.groupby(['pclass', 'family_size'])['fare'].transform('median'))

### Adding Other New Variables

In [ ]:
# Let's add some other variables while we're at it.

# Since fare tends to increase with family size, let's compute a fare per person
titanic['fare_per_person'] = titanic['fare'] / titanic['family_size']

# And let's add a variable to specify if a passenger is alone
titanic['is_alone'] = np.where(titanic['family_size'] == 1, 1, 0)

In [ ]:
# Visualize missing data again

plt.figure(figsize=(12, 8))
sns.heatmap(titanic.isnull(), cbar=False, cmap='viridis', yticklabels=False)
plt.title('Missing Values Heatmap in Titanic Dataset')
plt.xlabel('Columns')
plt.ylabel('Rows')
plt.show()

### Handling Missing Cabin Data

In [ ]:
# Now for the missing cabin data...

# Let's see what the cabin variable looks like

for cabin in titanic['cabin'].unique():
  print(cabin)

In [ ]:
# So some values of cabin have multiple entries...

# F and G occur together, as do E and F

# Let's clean this up by just taking the letter from the first entry in each cell

titanic['cabin'].str.split().str[0] # remove anything after a space
titanic['cabin'] = titanic['cabin'].fillna('').apply(lambda x: x[0] if x else None) # take only the first value (i.e., letter), if the value isn't 'None'

In [ ]:
# Let's see which cabins different passenger classes occupied
plt.figure(figsize=(12, 6))

# Create a count plot
sns.countplot(data=titanic, x='cabin', hue='pclass', palette='viridis', order=sorted(titanic['cabin'].dropna().unique()))

# Set plot titles and labels
plt.title('Distribution of Pclass for Different Cabin Categories')
plt.xlabel('Cabin Category')
plt.ylabel('Count')
plt.legend(title='Pclass')
plt.show()


# Cabins A, B, and C are only occupied by first-class passengers

In [ ]:
# What if we take the fare into account as well?

titanic['fare_per_person'] = titanic['fare'] / titanic['family_size']

titanic['fpp_bin'] = pd.qcut(titanic['fare_per_person'], 4, labels=[1,2,3,4])

# Set up the matplotlib figure
plt.figure(figsize=(10, 6))

# Create subplots for each fare_per_person_bin (fpp_bin)
fare_bins = sorted(titanic['fpp_bin'].unique())
for i, bin in enumerate(fare_bins, 1):
    plt.subplot(2, 2, i)
    sns.countplot(data=titanic[titanic['fpp_bin'] == bin], x='cabin', hue='pclass', palette='viridis', order=sorted(titanic['cabin'].dropna().unique()))
    plt.title(f'Pclass vs Cabin (Level {bin} Fare)')
    plt.xlabel('Cabin Category')
    plt.ylabel('Count')
    plt.legend(title='Pclass')

# Adjust layout
plt.tight_layout()
plt.show()


### Probabilistic Interpolation

In [ ]:
# Calculate the probabilities of each cabin given fare_per_person_bin and pclass
cabin_probs = titanic.dropna(subset=['cabin']).groupby(['fpp_bin', 'pclass', 'cabin']).size().unstack(fill_value=0)
cabin_probs = cabin_probs.div(cabin_probs.sum(axis=1), axis=0)
cabin_probs = cabin_probs.fillna(0)

# Display the cabin probabilities
print(cabin_probs)

In [ ]:
null_cabin_indices[0]

In [ ]:
null_cabin_indices = np.where(titanic.cabin.isnull())[0]

# Function to assign cabin probabilistically
def assign_cabin_probabilistically(row, cabin_probs):
    if pd.isna(row['cabin']) or row['cabin'] == '':
        fare_bin = row['fpp_bin']
        pclass = row['pclass']
        if (fare_bin, pclass) in cabin_probs.index:
            cabins = cabin_probs.loc[(fare_bin, pclass)]
            if cabins.max() == 0:
                # Fallback to probabilities for pclass only if all cabin probabilities are zero
                cabins = cabin_probs.groupby(level='pclass').sum().loc[pclass]
        else:
            # Fallback to probabilities for pclass only if the specific combination is not found
            cabins = cabin_probs.groupby(level='pclass').sum().loc[pclass]

        cabins = cabins[cabins > 0]  # Only consider non-zero probabilities
        return np.random.choice(cabins.index, p=cabins.values / cabins.values.sum())
    return row['cabin']

# Apply the function to assign missing cabin values
titanic['cabin_assigned'] = titanic.apply(assign_cabin_probabilistically, axis=1, cabin_probs=cabin_probs)

# View the result
print(titanic.loc[null_cabin_indices, ['pclass', 'fpp_bin', 'cabin_assigned']].head(30))

# fill in missing cabin values
titanic['cabin'] = titanic['cabin_assigned']

# drop extra columns we created along the way
titanic.drop(['cabin_assigned', 'fpp_bin'], axis = 1, inplace = True)

### Adding a Non-Categorical Cabin Variable

In [ ]:
# Cabins with higher letters (e.g., F and G) were deeper in the ship and possibly
# harder to exit, so let's assign integers for cabin level

from sklearn.preprocessing import LabelEncoder

# Encode cabin alphabetically
label_encoder = LabelEncoder()
titanic['cabin_int'] = label_encoder.fit_transform(titanic['cabin'])

In [ ]:
# Visualize missing data again (again)

plt.figure(figsize=(12, 8))
sns.heatmap(titanic.isnull(), cbar=False, cmap='viridis', yticklabels=False)
plt.title('Missing Values Heatmap in Titanic Dataset')
plt.xlabel('Columns')
plt.ylabel('Rows')
plt.show()

for col in titanic.columns:
  print(f'{col}: # missing = {titanic[col].isnull().sum()}')

# Much better

### More Variables!

In [ ]:
# Adding additional variables - is_married?

titanic['title'] = titanic['name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]
titanic['is_married'] = 0
titanic['is_married'].loc[titanic['title'] == 'Mrs'] = 1

# drop extra columns
titanic.drop(['title', 'name'], axis=1, inplace = True)

In [ ]:
# backup

titanic_backup = titanic.copy()
# titanic = titanic_backup.copy()

# Final Prep for ML

In [ ]:
# ML estimators don't like strings.

# For variables like 'sex' ('Male' vs. 'Female'), we need to do some form of dummy coding

# Typical dummy codes would assign a number for each level of sex (e.g., 1 for female, 2 for male)
# But most ML estimators will treat these dummy coded values linearly (male > female),
# which can be problematic.

# A common approach is 'One Hot Encoding', where we make a column for Male_yn,
# and a different column for Female_yn.

### One Hot Encoding for Categorical Variables

In [ ]:
# One hot encoding for sex variable

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Define the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), ['sex'])
    ])

# Define the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', LogisticRegression(max_iter=200))])


# Train/Test Split

In [ ]:
# Split the data into train and test sets
from sklearn.model_selection import train_test_split

X = titanic.drop(columns='survived')
y = titanic['survived'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# Refitting the Model

In [ ]:
# Base classifier

# Fit the pipeline
pipeline.fit(X_train, y_train)

# Evaluate the pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = pipeline.predict(X_test)

print("Base Model Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Plot confusion matrix
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - Base Model')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Further Improvements

### Feature Engineering

In [ ]:
# What's going on with age?

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml

# Define age bins and labels
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80]
labels = ['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80']
titanic['age_group'] = pd.cut(titanic['age'], bins=bins, labels=labels, right=False)

# Convert the 'survived' column to numeric
titanic['survived'] = pd.to_numeric(titanic['survived'])

# Calculate survival rate by age group
age_survival = titanic.groupby('age_group')['survived'].mean().reset_index()

# Plot the survival rate by age group
plt.figure(figsize=(10, 6))
sns.barplot(x='age_group', y='survived', data=age_survival, palette='viridis')
plt.title('Survival Rate by Age Group')
plt.xlabel('Age Group')
plt.ylabel('Survival Rate')
plt.ylim(0, 1)
plt.show()


In [ ]:
# This doesn't look linear. In fact, we can see a few changes in direction in
# the relationship between age and survival rate. This means the relationship is
# nonmonotonic. Two changes in direction mean a polynomial of order 3 should explain
# the relationship better than a linear relationship, but let's see if this checks out...

from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import KFold

# Load the Titanic dataset from scikit-learn
titanic_polytest = fetch_openml('titanic', version=1, as_frame=True)
titanic_polytest = titanic_polytest.frame

# Drop rows with missing target values and relevant features
titanic_polytest = titanic_polytest.dropna(subset=['survived', 'age'])

# Select relevant features and target
X = titanic_polytest[['age']]
y = titanic_polytest['survived']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Function to perform polynomial regression and return cross-validated RMSE
def polynomial_regression(degree, X_train, y_train):
    polynomial_features = PolynomialFeatures(degree=degree)
    X_train_poly = polynomial_features.fit_transform(X_train)

    model = LogisticRegression(max_iter=200)
    pipeline = Pipeline([('polynomial_features', polynomial_features),
                         ('scaler', StandardScaler()),
                         ('logistic_regression', model)])

    # Use cross-validation to evaluate the model
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='accuracy')

    # Return the mean of the cross-validation scores
    return np.mean(cv_scores)

# Evaluate polynomial regression models with different degrees
degrees = range(1, 6)
cv_scores = [polynomial_regression(degree, X_train, y_train) for degree in degrees]

# Plot the cross-validation scores for different polynomial degrees
plt.figure(figsize=(10, 6))
plt.plot(degrees, cv_scores, marker='o')
plt.title('Cross-Validation Accuracy for Polynomial Regression')
plt.xlabel('Polynomial Degree')
plt.ylabel('Cross-Validation Accuracy')
plt.show()

# Determine the best degree
best_degree = degrees[np.argmax(cv_scores)]
print(f'The best polynomial degree is {best_degree}')


# Final Touches

### Scaling the Data & Adding Interaction Terms

In [ ]:
# Identify numeric columns to be scaled
numeric_features = ['age', 'fare', 'family_size']

# Define the preprocessor to scale numeric features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features)
    ])

# Apply the preprocessor to scale the numeric features
titanic_scaled = preprocessor.fit_transform(titanic[numeric_features])

# Convert the scaled features back to a DataFrame
titanic_scaled_df = pd.DataFrame(titanic_scaled, columns=numeric_features)

# Compute the cubed age after scaling
titanic_scaled_df['age^3'] = titanic_scaled_df['age'] ** 3

# Create interaction terms using the scaled data
titanic_scaled_df['age^3*pclass'] = titanic_scaled_df['age^3'] * titanic['pclass']
titanic_scaled_df['fare*pclass'] = titanic_scaled_df['fare'] * titanic['pclass']
titanic_scaled_df['age^3*sex'] = titanic_scaled_df['age^3'] * (titanic['sex'] == 'male').astype(int)
titanic_scaled_df['sex*pclass'] = (titanic['sex'] == 'male').astype(int) * titanic['pclass']
titanic_scaled_df['family_size*pclass'] = titanic_scaled_df['family_size'] * titanic['pclass']

# Combine the scaled and interaction terms with other relevant features
titanic_prepared = pd.concat([titanic[['pclass', 'sex', 'cabin']], titanic_scaled_df], axis=1)


### Feature Selection, Model Tuning, & Cross Validation

In [ ]:
# Define the preprocessor for categorical and numeric data
full_preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['pclass', 'sex', 'cabin']),
        ('num', 'passthrough', titanic_scaled_df.columns.tolist())
    ])

# Define the pipeline with RFE and logistic regression
pipeline = Pipeline(steps=[
    ('preprocessor', full_preprocessor),
    ('feature_selection', RFE(estimator=LogisticRegression(max_iter=500))),
    ('classifier', LogisticRegression(max_iter=200))
])

# Split the data into features and target
X = titanic_prepared
y = titanic['survived'].astype(int)

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'feature_selection__n_features_to_select': [5,6,7,8,9,10],  # Number of top features to select
    'classifier__C': [0.01, 0.1, 1, 10, 100],
    'classifier__solver': ['liblinear', 'saga']
}

# Define the GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, scoring='accuracy')

# Fit the GridSearchCV
grid_search.fit(X, y)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Cross-Validation Score:", best_score)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_

# Split the data into train and test sets for evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

print("Test Set Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Plot the confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

# Plot the ROC curve
y_pred_proba = best_model.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = roc_auc_score(y_test, y_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC Curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()

# Get the feature selection step
feature_selection = best_model.named_steps['feature_selection']

# Get the mask of selected features
selected_features_mask = feature_selection.support_

# Get the original feature names
ohe_feature_names = best_model.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(['pclass', 'sex', 'cabin']).tolist()
num_feature_names = titanic_scaled_df.columns.tolist()

# Combine feature names
feature_names = ohe_feature_names + num_feature_names

# Get the selected and excluded features
selected_features = [feature for feature, selected in zip(feature_names, selected_features_mask) if selected]
excluded_features = [feature for feature, selected in zip(feature_names, selected_features_mask) if not selected]

print("Selected Features:", selected_features)
print("Excluded Features:", excluded_features)
